In [ ]:
import cv2, threading, time
class VIDEO():# image cache
    def __init__(self):  
        self.frame = b''
        self.cap = cv2.VideoCapture("tcp://127.0.0.1:2222") 
        
    def close(self):
        self.cap.release()
        
    def updater(self):   
        while(self.cap.isOpened()):  
            ret, frame = self.cap.read() 
            if not ret: 
                print('continue')
                continue 
            ret, jpeg  = cv2.imencode('.jpg', frame)
            self.frame = jpeg.tobytes() 
            
video = VIDEO()
video_link_thread = threading.Thread(target=video.updater)
video_link_thread.start() 

In [ ]:
import flask, random
app = flask.Flask(__name__)
print('Click Here and see the video: http://127.0.0.1:9999/stream.mjpg\n\n\n')
fps=25
interval = 1/fps
@app.route("/stream.mjpg") 
def mjpg1():  
    def generator():     
        id_ =  random.random()
        while True:     
            time.sleep(interval)  # threading.condition is too shitty according to my test. no condition no lag. 
            frame = video.frame  
            yield f'''--FRAME\r\nContent-Type: image/jpeg\r\nContent-Length: {len(frame)}\r\n\r\n'''.encode() 
            yield frame
    r = flask.Response(response=generator(), status=200)
    r.headers.extend({'Age':0, 'Content-Type':'multipart/x-mixed-replace; boundary=FRAME',
                      'Pragma':'no-cache', 'Cache-Control':'no-cache, private',}) 
    return r

@app.route('/')
def hello_world():
    return 'Hello, World!'

app.run('127.0.0.1', 9999)